In [1]:
import database
import core

<h1> Character Mapping </h1>

<h2><b>database.SaveMap</b>(<i>name, transform, inverse, keywords</i>)</h2>

<DL>
<DT><i>name</i>
<DD>The name of the cipher to be added to the database. Will raise an exception the passed name is already taken.
<DT><i>transfom</i>
<DD>A string consisting of valid python that transforms plaintext into cypher text. The locals <i>maskedIndices</i> and <i>mapRange</i>, which are generated in the mapping process, are supplied to the formula during execution. See <b>MAPPING_NOTEBOOK</b> for more. Additionally, <i>numpy</i> is automatically imported as <i>np</i> at run time. The names <i>__func_name__</i> , <i>__var_name__</i> , and <i>__class_instance__</i> are reserved and will raise exceptions if used within the formula's text.
<DT><i>inverse</i>
<DD> if inverse is <i>None</i>, then one will be generated.
<DT><i>keywords</i>
<DD>A list containing keywords that can be sorted by once cipher is added to database.
</DL>

In [4]:
if len(database.con.run("""SELECT 1 FROM maps WHERE "name"='alphaLower'""")) == 0:
    transform = {
        "A":0,  "a":0,  "B":1,  "b":1,  "C":2,  "c":2,  "D":3,  "d":3,  "E":4,  "e":4,
        "F":5,  "f":5,  "G":6,  "g":6,  "H":7,  "h":7,  "I":8,  "i":8,  "J":9,  "j":9,
        "K":10, "k":10, "L":11, "l":11, "M":12, "m":12, "N":13, "n":13, "O":14, "o":14,
        "P":15, "p":15, "Q":16, "q":16, "R":17, "r":17, "S":18, "s":18, "T":19, "t":19,
        "U":20, "u":20, "V":21, "v":21, "W":22, "w":22, "X":23, "x":23, "Y":24, "y":24,
        "Z":25, "z":25}

    t = core.DecompressTransform(transform)[0]
    i = core.GenInverseTransform(t)

    inverse = core.CompressTransform(i)

    keywords = ["alpha", "lower", "ascii", "alphabet"]

    database.SaveMap("alphaLower", transform, inverse, keywords)

TypeError: Object of type int64 is not JSON serializable

<h1> Adding, Accessing and Using Ciphers </h1>

<h2><b>database.SaveCipher</b>(<i>name, formula, inverse, keywords, options</i>)</h2>

Security checks are automatically preformed whenever this function is called.
Notably, exceptions will be raised if any security check fails. Additional
information is given in
<b>SECURITY_NOTEBOOK</b>

<DL>
<DT><i>name</i>
<DD>The name of the cipher to be added to the database. Will raise an exception the passed name is already taken.
<DT><i>formula</i>
<DD>A string consisting of valid python code that transforms a mapped plaintext array into cypher text. The locals <i>maskedIndices</i>, <i>mappedIndices</i> and <i>mapRange</i> are generated in the mapping process and then supplied to the formula during execution. See <b>MAPPING_NOTEBOOK</b> for more. Additionally, <i>numpy</i> is automatically imported as <i>np</i> at run time. The names <i>__func_name__</i> , <i>__var_name__</i> , and <i>__class_instance__</i> are reserved and will raise exceptions if used within the formula's text.
<DT><i>inverse</i>
<DD>Python code that transforms cypher text into plaintext. The rules for the <i>formula</i> parameter apply.
<DT><i>keywords</i>
<DD>A list containing keywords that can be sorted by once cipher is added to database.
<DT><i>options</i>
<DD>A dictionary in which each key/value pair represents the name and default value of extra locals passed to cipher/inverse. 
</DL>

In [2]:
# NOTE: This only runs if a vigenere cipher is not aready in the system
if len(database.con.run("""SELECT 1 FROM ciphers WHERE "name"='vigenere'""")) == 0:

    keywords = ["vigenere","caesar","polyalphabetic"]
    options = {"cycleKeywordOutsideMap":False, "deleteTextOutsideMap":True}

    formulaStr = """
    assert len(mapRange) == 1 + max(mapRange)

    if options["cycleKeywordOutsideMap"]:
        offset = np.resize(keys[0], len(text))[mappedIndices]

    else:
        offset = np.resize(keys[0], len(mappedIndices))

    out = (text[mappedIndices] + offset) % len(mapRange)

    if not options["deleteTextOutsideMap"]:
        out = np.put(text, mappedIndices, out)
    """

    inverseStr = """
    assert len(mapRange) == 1 + max(mapRange)

    if options["cycleKeywordOutsideMap"]:
        offset = np.resize(keys[0], len(text))[mappedIndices]
    else:
        offset = np.resize(keys[0], len(mappedIndices))

    if options["deleteTextOutsideMap"]:
        out = (text - offset) % len(mapRange)
    else:
        out = (text[mappedIndices] - offset) % len(mapRange)
        out = np.put(text, mappedIndices, out)
    """

    database.SaveCipher("vigenere", formulaStr, inverseStr, keywords, options)

<h2><b>database.LoadCipher</b>(<i>identifier</i>)</h2>

<DL>
<DT><i>identifier</i>
<DD>This can be either a sting or an int. If identifier is an integer, <i>LoadCipher( )</i> will return the cipher with that ID. Otherwise, <i>LoadCipher( )</i> will return all ciphers whose name matches the passed identifier.
</DL>

In [3]:
query  = database.LoadCipher("vigenere")
print(f"Hash of first cipher: '{query[1]}'")


Hash of first cipher: '[-679745061, 1038564999, -428354572, -412549081, -2071110137, -1827022449, 2001695571, 638559363, 595319930, 1491456457, 571760906, 1964182341, 744083102, -1494954262, 1632359335, -1648850155]'
